In [3]:
q = 11
F = GF(q)
print(F)
g=2
n, k = 11, 5
C = codes.GeneralizedReedSolomonCode(F.list()[:n], k)
G = C.generator_matrix()
H = C.parity_check_matrix()
D = C.decoder()

Finite Field of size 11


In [4]:
print(G)

[ 1  1  1  1  1  1  1  1  1  1  1]
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  4  9  5  3  3  5  9  4  1]
[ 0  1  8  5  9  4  7  2  6  3 10]
[ 0  1  5  4  3  9  9  3  4  5  1]


In [5]:
print(H)

[10 10 10 10 10 10 10 10 10 10 10]
[ 0 10  9  8  7  6  5  4  3  2  1]
[ 0 10  7  2  6  8  8  6  2  7 10]
[ 0 10  3  6  2  7  4  9  5  8  1]
[ 0 10  6  7  8  2  2  8  7  6 10]
[ 0 10  1 10 10 10  1  1  1 10  1]


In [6]:
print(C.minimum_distance())
t=(C.minimum_distance()-1)//2
print(t)

7
3


In [7]:
def getvector(vec,number,k):
    for i in range(k):
        temp=pow(2,k-1-i)
        if temp<=number:
            vec[i]=1
            number-=temp
        else:
            vec[i]=0
    return vec
def swap_cols(M,col1,col2,dim):
    for i in range(dim):
        if M[i][col1]==1:
            temp1=i
        if M[i][col2]==1:
            temp2=i 
    M[temp1][col2]=1
    M[temp2][col2]=0
    M[temp2][col1]=1
    M[temp1][col1]=0
    return M

In [8]:
S_=[[0 for y in range(n-k)] for x in range(n-k)]
for i in range(n-k):
    for j in range(n-k):
        if (i==j):
            S_[i][j]=1
        elif (i<j):
            S_[i][j]=randint(0,q-1)
        else:
            S_[i][j]=0

In [9]:
S=Matrix(GF(q),S_)
print(S)
print()
s_inv = S.inverse()
print(s_inv)


[ 1  5  3  0 10  7]
[ 0  1  8 10  1  3]
[ 0  0  1  7  9  7]
[ 0  0  0  1  8  1]
[ 0  0  0  0  1  4]
[ 0  0  0  0  0  1]

[ 1  6  4  0  3  1]
[ 0  1  3  2  0  7]
[ 0  0  1  4  3 10]
[ 0  0  0  1  3  9]
[ 0  0  0  0  1  7]
[ 0  0  0  0  0  1]


In [10]:
P_=[[0 for y in range(n)] for x in range(n)]
for i in range(n):
    for j in range(n):
        if (i==j):
            P_[i][j]=1
P=Matrix(GF(q),P_)
for i in range(len(P_)):
    k1=randint(0,len(P_)-1)
    k2=randint(0,len(P_)-1)
    if k2!=k1:
        swap_cols(P_,k1,k2,len(P_))
P=Matrix(GF(q),P_)
print(P)
p_inv=P.inverse()

[0 0 0 0 0 1 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0]


In [11]:
H_pub = S*H*P
print(H_pub)
H_pub_t = H_pub.transpose()
print(H_pub_t)

[ 7  3  0  6  4 10  0  9  9  9  9]
[10 10  5  9  3  0  7  4  0  5  2]
[ 9  5  1  0  8  0  2  6  1  4  8]
[ 1  8  8  0 10  0  0 10  6  5  7]
[ 6  1 10  2  4  0  9  6  3  3  0]
[10  1  1 10 10  0 10  1 10  1  1]
[ 7 10  9  1  6 10]
[ 3 10  5  8  1  1]
[ 0  5  1  8 10  1]
[ 6  9  0  0  2 10]
[ 4  3  8 10  4 10]
[10  0  0  0  0  0]
[ 0  7  2  0  9 10]
[ 9  4  6 10  6  1]
[ 9  0  1  6  3 10]
[ 9  5  4  5  3  1]
[ 9  2  8  7  0  1]


In [92]:
message = vector([0 for y in range(n)])
for i in range(t):
    temp = randint(0,n-1)
    if message[temp]!=0:
        while True:
            temp = randint(0,n-1)
            if message[temp]!=0:
                break
    message[temp]=randint(1,n-1)
print('message',message)

message (9, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0)


In [93]:
cipher = message*H_pub_t
print(cipher)

(9, 4, 4, 6, 6, 7)


In [94]:
import numpy 
def GRS_sindrom_decoder(c,s_inv,p_inv,code,t,q,g):
    s = c*transpose(s_inv)
    print('Синдромы',s)
    det = 0
    while det == 0:
        M=([[0 for y in range(t)] for x in range(t)])
        for i in range(t):
            for j in range(t):
                M[i][j] =s[i+j]
        M_res = Matrix(GF(q),M)
        det = M_res.det()
        v = t
        t-=1
    print(M_res)
    print('Определитель',det)
    print('Ошибок -',v)
    M_res = M_res.inverse()
    print(M_res)
    print('Mv^-1')
    M_res=M_res.transpose()
    print(M_res)
    delta =0
    print(-s[v:2*v])
    delta = -s[v:2*v]*M_res
    print('delta',delta)
    x = PolynomialRing(GF(q), 'x').gen()
    delta_pol = 0
    for i in range(0, v):
            delta_pol += delta[i]*x^(v-i)
    print('delta',delta)
    delta_pol+=1
    print('Многочлен локаторов', delta_pol)
    roots = [r for r, i in delta_pol.roots()]
    print('Корни',roots)
    roots_= [r^-1 for r, i in delta_pol.roots()]
    print('roots_',roots_)
    system = matrix(GF(q), [[roots_[i]^j for i in range(v)] for j in range(v)])
    y = system.solve_right(s[0:v])
    print(system)
    print('Значения ошибок',y)
    e = vector(GF(q),n)
    for i in range(v):
        e[roots_[i]] = -y[i]
    print('Вектор ошибки',e)
    e = e * transpose(p_inv)
    print(e)
    

In [95]:
GRS_sindrom_decoder(cipher,s_inv,p_inv,C,t,q,g)

Синдромы (8, 0, 6, 10, 0, 7)
[ 8  0  6]
[ 0  6 10]
[ 6 10  0]
Определитель 7
Ошибок - 3
[ 3  7  9]
[ 7  9  9]
[ 9  9 10]
Mv^-1
[ 3  7  9]
[ 7  9  9]
[ 9  9 10]
(1, 0, 4)
delta (6, 10, 5)
delta (6, 10, 5)
Многочлен локаторов 6*x^3 + 10*x^2 + 5*x + 1
Корни [10, 2, 1]
roots_ [10, 6, 1]
[ 1  1  1]
[10  6  1]
[ 1  3  1]
Значения ошибок (7, 10, 2)
Вектор ошибки (0, 9, 0, 0, 0, 0, 1, 0, 0, 0, 4)
(9, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0)
